In [9]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import numpy as np
import os

import ray
from ray import tune
from ray.air import session
from ray.air.checkpoint import Checkpoint
from ray.tune.schedulers import ASHAScheduler
from ray.tune import CLIReporter

from sklearn.model_selection import StratifiedKFold
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import Dataset

#####
import pandas as pd
import numpy as np
import os
from functools import reduce
from numpy import squeeze
from torch.utils.data import Dataset
from torch import Tensor, vstack, cat, mean, std, torch
####
from all_data_dataset import all_data_dataset
from hyperparm_tunning_new_tune import tuner, train
from train_test_no_tune import train_test_no_tune

In [7]:
paths = {
    'path_gwas' : os.path.dirname('~/fast/pd_subtype/data/ad_genes_OR.csv'),
    'fn_gwas' : os.path.basename('~/fast/pd_subtype/data/ad_genes_OR.csv'),
    'path_snp' : os.path.dirname('../../../../bulk/machad/ADNI/geno/final/adni_wgs_recode.raw'),
    'fn_snp' : os.path.basename('../../../../bulk/machad/ADNI/geno/final/adni_wgs_recode.raw'),
    'path_clin' : os.path.dirname('../../../../bulk/machad/ADNI/ADNIMERGE.csv'),
    'fn_clin' : os.path.basename('../../../../bulk/machad/ADNI/ADNIMERGE.csv'),
    'path_img' : os.path.dirname('../../../../bulk/machad/ADNI/imaging/TADPOLE_D1_D2.csv'),
    'fn_img' : os.path.basename('../../../../bulk/machad/ADNI/imaging/TADPOLE_D1_D2.csv'),
    'path_img_feat':os.path.dirname('../../../../bulk/machad/ADNI/adni_img_feat_names_crossectional.txt'),
    'fn_img_feat': os.path.basename('../../../../bulk/machad/ADNI/adni_img_feat_names_crossectional.txt'),
    'path_labels':os.path.dirname('~/fast/pd_subtype/results/figures/adni_labels.csv'),
    'fn_labels': os.path.basename('~/fast/pd_subtype/results/figures/adni_labels.csv'),
    'path_feat_dict' : os.path.dirname('../../../../bulk/machad/ADNI/TADPOLE_D1_D2_Dict.csv'),
    'fn_feat_dict' : os.path.basename('../../../../bulk/machad/ADNI/TADPOLE_D1_D2_Dict.csv'), 
    'path_res' : '../results',
    'checkpoint_name' : '../checkpoints'
}
args = {
    'demo_vars' : ['PTID','AGE','PTGENDER','PTETHCAT','PTRACCAT', 'MMSE'],
#     'clin_vars' : ['ABETA_bl_UCSFFSL_02_01_16_UCSFFSL51_03_01_22', 'TAU_bl_UCSFFSL_02_01_16_UCSFFSL51_03_01_22', 'PTAU_bl_UCSFFSL_02_01_16_UCSFFSL51_03_01_22', 'FDG_bl_UCSFFSL_02_01_16_UCSFFSL51_03_01_22'],
    'clin_vars' : ['RAVLT_immediate' ,'RAVLT_learning' ,'RAVLT_forgetting' ,'RAVLT_perc_forgetting' ,'LDELTOTAL', 'DIGITSCOR', 'TRABSCOR'],
    'outcome_vars' : ['MMSE','CDRSB'],
    'visits' : ['bl','m06','m12','m24'],
    'impute': True,
    'visit_colnm' : 'VISCODE',
    'tune_flag':False,
    'ablation':'rf',
    'use_cluster_flag':True
}


In [8]:
all_inputs = all_data_dataset(paths, args)

/fast/machad/pd_subtype/released/all_data_dataset.py:100: DtypeWarning: Columns (22,23,24,53,54,82,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df_full = pd.read_csv(os.path.join(self.path,self.fn))


In [ ]:
def full_exp(paths, args, config):
    results_dict_all_splits = {}

    # Load data
    all_inputs = all_data_dataset(paths, args)
   
    # Generate splits using sklearn
    skf_outer = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
    # for i, (train_index, test_index) in enumerate(skf_outer.split(all_inputs[:][0], all_inputs[:][3])):
    for i, (train_index, test_index) in enumerate(skf_outer.split(np.zeros(len(all_inputs.get_labels())), all_inputs.get_labels())):
        results_dict_per_test_set = {}
        skf_inner = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
        # for j, (train_index_sub, val_index) in enumerate(skf_inner.split(train_index, all_inputs[train_index][3])):
        for j, (train_index_sub, val_index) in enumerate(skf_inner.split(train_index, all_inputs.get_labels(train_index))):
            train_index_sub = train_index[train_index_sub]
            val_index = train_index[val_index]
            assert np.intersect1d(train_index_sub, val_index).size == 0 and np.intersect1d(train_index_sub, test_index).size == 0 and np.intersect1d(val_index, test_index).size == 0 
            # Hyperparam tunning (Train - validate)
            all_inputs.set_trainset_mean_std(train_index)
            all_inputs.normalize_data(train_index_sub, val_index, test_index)
            if args['tune_flag']:
                results_dict_per_test_set[j] = tuner(all_inputs, train_index_sub, val_index, test_index, paths, args)
            else:
                results_dict_per_test_set[j] = train_test_no_tune(all_inputs, train_index_sub, val_index, test_index, paths, args, config)
        
        print('#'*40)
        print(f'Finished 5-fold cv on test set: {i}')

        results_dict_all_splits[i] = results_dict_per_test_set
        
    print('#'*40)
    print('Finished 10-fold cross testing on test set')
    return results_dict_all_splits

In [ ]:
metrics =[]
for i in range(10):
    for j in range(5):
        metrics.append(pd.DataFrame.from_dict([results_dict_all_splits[i][j]['metrics']]))

# TODO: concatenate the metrics dataframe per iteration into 1
merged_df = pd.concat(metrics)
